In [1]:
import open3d as o3d
import numpy as np
from collections import deque
import pandas as pd

In [2]:
pcd = o3d.io.read_point_cloud("/Users/deeprodge/Downloads/pbr28.pcd")

In [3]:
class RegionGrowingSegmentation:
    def __init__(self, pcd, num_neighbors=30, smoothness_threshold=np.pi/4):
        self.pcd = pcd
        self.num_neighbors = num_neighbors
        self.smoothness_threshold = smoothness_threshold
        self.labels = np.array([-1] * len(pcd.points))  # -1 indicates unlabeled
        self.normals = np.asarray(pcd.normals)
        self.pcd_tree = o3d.geometry.KDTreeFlann(self.pcd)

    def grow_region(self, starting_indices):
        for region_index, starting_index in enumerate(starting_indices):
            if self.labels[starting_index] != -1:  # If already labeled, skip
                continue

            queue = deque([starting_index])
            self.labels[starting_index] = region_index

            while queue:
                current_index = queue.popleft()
                current_normal = self.normals[current_index]

                [k, idx, _] = self.pcd_tree.search_knn_vector_3d(self.pcd.points[current_index], self.num_neighbors)

                for neighbor_index in idx[1:]:  # Skip the first index because it's the point itself
                    if self.labels[neighbor_index] != -1:
                        continue

                    angle = np.arccos(np.clip(np.dot(current_normal, self.normals[neighbor_index]), -1.0, 1.0))

                    if angle < self.smoothness_threshold:
                        self.labels[neighbor_index] = region_index
                        queue.append(neighbor_index)

    def segment(self):
        # Choose the lowest point as terrain seed
        z_values = np.array(self.pcd.points)[:, 2]
        terrain_seed = np.argmin(z_values)

        # Choose a high point with potentially fewer neighbors as rock seed
        potential_rock_seeds = np.where(z_values > np.percentile(z_values, 75))[0]
        rock_seed = potential_rock_seeds[np.argmax(z_values[potential_rock_seeds])]
        print(terrain_seed)
        print(rock_seed)

        # Temp code to check and locate seed in the point cloud
        # self.labels[terrain_seed] = 1
        # self.labels[rock_seed] = 1

        self.grow_region([terrain_seed, rock_seed])

        # Assign remaining unlabeled points to the nearest labeled point
        unlabeled_points = np.where(self.labels == -1)[0]
        for unlabeled_point in unlabeled_points:
            [_, idx, _] = self.pcd_tree.search_knn_vector_3d(self.pcd.points[unlabeled_point], 1)
            self.labels[unlabeled_point] = self.labels[idx[0]]

        # Now, set any remaining unlabeled points to a default label (0 or 1). This should be rare.
        self.labels[self.labels == -1] = 1

        return self.labels


In [4]:
# Load point cloud
pcd = o3d.io.read_point_cloud("/Users/deeprodge/Downloads/pbr28.pcd")

# Preprocess the point cloud to estimate normals
pcd.estimate_normals(search_param=o3d.geometry.KDTreeSearchParamHybrid(radius=0.5, max_nn=100))


In [6]:
# Create an instance of the RegionGrowingSegmentation class
rgs = RegionGrowingSegmentation(pcd, num_neighbors=20, smoothness_threshold=np.pi/5)

# Perform region growing segmentation
segment_labels = rgs.segment()

23401
57596


In [7]:
np.unique(segment_labels)

array([0, 1])

In [8]:
# Assign colors to points for each segment (blue for terrain, red for rock)
colors = np.array([[0, 0, 1] if label == 0 else [1, 0, 0] for label in segment_labels])
pcd.colors = o3d.utility.Vector3dVector(colors)

In [11]:
# Show the segmented point cloud
o3d.visualization.draw_geometries([pcd])

[Open3D WARNING] GLFW Error: Cocoa: Failed to find service port for display
